In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_rows', 50)
import tabula
from pandasql import sqldf
import math
from scipy.stats import chi2
from scipy import stats
from matplotlib import pyplot as plt

In [7]:
epigenomics = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\metaData_soybean_drought.xlsx', sheet_name = 'DNA')

In [14]:
epi_data = epigenomics[epigeomics['omics_type'].str.contains('Epigenomic')].reset_index().drop(['index'], axis = 1)

In [23]:
epi_data_mapped = epi_data[epi_data['mapped_gene'].isna()].reset_index().drop(['index'], axis = 1)[['marker', 'mapped_gene', 'chr', 'bp0', 'bp1', 'fc', 'drought_treatment_span', 'article_name', 'gene_version']]
epi_data_already = epi_data[epi_data['mapped_gene'].isna() == False].reset_index().drop(['index'], axis = 1)[['mapped_gene', 'fc', 'drought_treatment_span', 'article_name', 'gene_version']]

In [44]:
glyma_gene_ver = pd.read_csv(r'C:\Users\edwar\Desktop\農藝\paper2\Glyma_genes_correspondence_final.txt', delimiter = '\t')

In [24]:
epi_data_mapped

,marker,mapped_gene,chr,bp0,bp1,fc,drought_treatment_span,article_name,gene_version
0,mir166a-5p,NaN,16,1912570.0,1912715.0,1.50,2.5,Identification of novel soybean microRNAs invo...,1.0
1,mir166f,NaN,20,43105394.0,43105500.0,0.71,2.5,Identification of novel soybean microRNAs invo...,1.0
2,mir169f-3p,NaN,20,46876643.0,46876727.0,1.40,2.5,Identification of novel soybean microRNAs invo...,1.0
3,mir1513c,NaN,20,223679.0,223766.0,1.80,2.5,Identification of novel soybean microRNAs invo...,1.0
4,mir4415b,NaN,8,23142767.0,23142922.0,2.20,2.5,Identification of novel soybean microRNAs invo...,1.0
5,mir-seq07,NaN,1,44787899.0,44787988.0,1.50,2.5,Identification of novel soybean microRNAs invo...,1.0
6,mir-seq11,NaN,16,5744795.0,5744863.0,1.87,2.5,Identification of novel soybean microRNAs invo...,1.0
7,mir-seq13,NaN,4,28578972.0,28579075.0,1.75,2.5,Identification of novel soybean microRNAs invo...,1.0


only gene version 1.0 and one article

In [25]:
# gene mapping
template_a1v10 = pd.read_csv(r'C:\Users\edwar\Desktop\農藝\paper2\Glyma1.0_mapping.txt', delimiter = '\t')
template_a1v10['bp0'] = template_a1v10['bp0'].apply(lambda x : x - 20000)
template_a1v10['bp1'] = template_a1v10['bp1'].apply(lambda x : x + 20000)

def gene_mapping_ver(df, template, i):
    cri1 = (df.iloc[i]['bp0'] - template['bp0']) * (df.iloc[i]['bp0'] - template['bp1']) <= 0
    cri2 = (df.iloc[i]['bp1'] - template['bp1']) * (df.iloc[i]['bp1'] - template['bp0']) <= 0
    cri3 = (df.iloc[i]['bp1'] < template['bp1']) & (df.iloc[i]['bp0'] > template['bp0'])
    cri4 = (df.iloc[i]['chr'] == template['chr'])
    return template[((cri1 | cri2) | cri3) & (cri4)].reset_index().drop(['index'], axis = 1)

In [29]:
i = 0
gene_mapping_ver(epi_data_mapped, template_a1v10, i)
epi_data_mapped.iloc[i]['fc']

1.5

In [30]:
gene_mapped_data = pd.DataFrame()

for i in range(len(epi_data_mapped['mapped_gene'])):
    df_gene_v1 = pd.DataFrame()
    df_gene = gene_mapping_ver(epi_data_mapped, template_a1v10, i)
    df_gene_v1['gene'] = df_gene['gene']
    df_gene_v1['fc'] = epi_data_mapped.iloc[i]['fc']
    df_gene_v1['gene_version'] = epi_data_mapped.iloc[i]['gene_version']
    gene_mapped_data = pd.concat([gene_mapped_data, df_gene_v1]).reset_index().drop(['index'], axis = 1)
    
    print(i)

0
1
2
3
4
5
6
7


In [32]:
gene_mapped_data.loc[(gene_mapped_data['fc'] < 1), 'fc'] = 1 / gene_mapped_data['fc']

In [38]:
epi_data_already = epi_data_already.rename(columns = {'mapped_gene':'gene'})

In [40]:
gene_data_all = pd.concat([gene_mapped_data, epi_data_already[['gene', 'fc', 'gene_version']]]).reset_index().drop(['index'], axis = 1)

In [51]:
gene_data_all = gene_data_all.merge(glyma_gene_ver[['Wm82.a2.v1', 'Wm82.a1.v1']], how = 'left', left_on = ['gene'], right_on = ['Wm82.a1.v1']).drop(['Wm82.a1.v1'], axis = 1).rename(columns = {'gene':'gene_org', 'Wm82.a2.v1':'gene'})

In [55]:
gene_data_all_v1 = gene_data_all[gene_data_all['gene'].isna() == False].drop(['gene_version', 'gene_org'], axis = 1).reset_index().drop(['index'], axis = 1)[['gene', 'fc']]

In [57]:
gene_data_all_v1['score'] = gene_data_all_v1['fc']

In [59]:
gene_data_all_v1['sig_flag'] = ''
gene_data_all_v1['nonsig_flag'] = ''
gene_data_all_v1.loc[(gene_data_all_v1['fc'] > 1.5), 'sig_flag'] = 1
gene_data_all_v1.loc[(gene_data_all_v1['fc'] > 1.5), 'nonsig_flag'] = 0
gene_data_all_v1.loc[(gene_data_all_v1['fc'] <= 1.5), 'sig_flag'] = 0
gene_data_all_v1.loc[(gene_data_all_v1['fc'] <= 1.5), 'nonsig_flag'] = 1

In [61]:
gene_data_all_v1.to_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\epigenomics_fv.xlsx', index = False)